### Imports and Get Data

In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer

In [16]:
df = pd.read_csv('train.csv') #change path to yours
X = df.loc[:, df.columns != 'Survived']
y = df['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [17]:
X_train.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
298,299,1,"Saalfeld, Mr. Adolphe",male,NaN,0,0,19988,30.5000,C106,S
884,885,3,"Sutehall, Mr. Henry Jr",male,25.00,0,0,SOTON/OQ 392076,7.0500,NaN,S
247,248,2,"Hamalainen, Mrs. William (Anna)",female,24.00,0,2,250649,14.5000,NaN,S
478,479,3,"Karlsson, Mr. Nils August",male,22.00,0,0,350060,7.5208,NaN,S
305,306,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.5500,C22 C26,S


---
---



### Code Snippet 1

In [18]:
# def famsize(df):
#     df['Family_size'] = df['SibSp'] + df['Parch']
#     df['Family_size_cat'] = np.where(df.Family_size == 0, 1, np.where( 
#     df.Family_size == 1, 2, np.where(
#     df.Family_size == 2, 2, np.where( 
#     df.Family_size == 3, 3, np.where( 
#     df.Family_size > 3, 4, 0)))))
#     return df['Family_size_cat'].value_counts()

def famsize(df):
    df = df.copy()
    df['Family_size'] = df['SibSp'] + df['Parch']
    df['Family_size_cat'] = np.where(df.Family_size == 0, 1, np.where( 
    df.Family_size == 1, 2, np.where(
    df.Family_size == 2, 2, np.where( 
    df.Family_size == 3, 3, np.where( 
    df.Family_size > 3, 4, 0)))))
    return df['Family_size_cat'].value_counts()

In [19]:
famsize(X_train)

1    406
2    190
4     47
3     25
Name: Family_size_cat, dtype: int64

In [20]:
X_train

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
298,299,1,"Saalfeld, Mr. Adolphe",male,NaN,0,0,19988,30.5000,C106,S
884,885,3,"Sutehall, Mr. Henry Jr",male,25.00,0,0,SOTON/OQ 392076,7.0500,NaN,S
247,248,2,"Hamalainen, Mrs. William (Anna)",female,24.00,0,2,250649,14.5000,NaN,S
478,479,3,"Karlsson, Mr. Nils August",male,22.00,0,0,350060,7.5208,NaN,S
305,306,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.5500,C22 C26,S
...,...,...,...,...,...,...,...,...,...,...,...
106,107,3,"Salkjelsvik, Miss. Anna Kristine",female,21.00,0,0,343120,7.6500,NaN,S
270,271,1,"Cairns, Mr. Alexander",male,NaN,0,0,113798,31.0000,NaN,S
860,861,3,"Hansen, Mr. Claus Peter",male,41.00,2,0,350026,14.1083,NaN,S
435,436,1,"Carter, Miss. Lucile Polk",female,14.00,1,2,113760,120.0000,B96 B98,S


In [ ]:
def famsize(df):
    df = df.copy()
    df['Family_size'] = df['SibSp'] + df['Parch']
    df['Family_size_cat'] = np.where(df.Family_size == 0, 1, np.where( 
    df.Family_size == 1, 2, np.where(
    df.Family_size == 2, 2, np.where( 
    df.Family_size == 3, 3, np.where( 
    df.Family_size > 3, 4, 0)))))
    return df['Family_size_cat'].value_counts()

### Code Snippet 2

In [5]:
def k_binner(df, col_name):
    kbins = KBinsDiscretizer(n_bins=5, encode='onehot', strategy='uniform')
    columns = df[[col_name]]
    kbins.fit(columns)
    t = kbins.transform(columns)
    t = t.todense()  

    # create nice labels
    edges = kbins.bin_edges_[0].round(1)
    labels = []
    for i in range(len(edges)-1):
        edge1 = edges[i]
        edge2 = edges[i+1]
        labels.append(f"{col_name} {edge1} to {edge2}")

    #create a DataFrame
    df_bins = pd.DataFrame(t, columns=labels)

    return df_bins

In [6]:
k_binner(X_train.copy(), 'SibSp').head(1)

,SibSp 0.0 to 1.6,SibSp 1.6 to 3.2,SibSp 3.2 to 4.8,SibSp 4.8 to 6.4,SibSp 6.4 to 8.0
0,1.0,0.0,0.0,0.0,0.0


In [7]:
k_binner(X_test.copy(), 'SibSp').head(1)

,SibSp 0.0 to 0.8,SibSp 0.8 to 1.6,SibSp 1.6 to 2.4,SibSp 2.4 to 3.2,SibSp 3.2 to 4.0
0,0.0,1.0,0.0,0.0,0.0


### Code Snippet 3

In [8]:
def deck(X_train):
    return pd.DataFrame(X_train['Cabin'].str[0])

In [9]:
ft = FunctionTransformer(deck)

In [10]:
ft.fit_transform(X_train['Cabin'])

KeyError: 'Cabin'

### Code Snippet 4

In [11]:
def Name_Title(X_train):
    X_train['Name'] = X_train.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())
    return X_train

In [12]:
Name_Title(X_train)

<ipython-input-11-48c65d3fea50>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['Name'] = X_train.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Family_size,Family_size_cat
298,299,1,Mr,male,NaN,0,0,19988,30.5000,C106,S,0,1
884,885,3,Mr,male,25.00,0,0,SOTON/OQ 392076,7.0500,NaN,S,0,1
247,248,2,Mrs,female,24.00,0,2,250649,14.5000,NaN,S,2,2
478,479,3,Mr,male,22.00,0,0,350060,7.5208,NaN,S,0,1
305,306,1,Master,male,0.92,1,2,113781,151.5500,C22 C26,S,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,107,3,Miss,female,21.00,0,0,343120,7.6500,NaN,S,0,1
270,271,1,Mr,male,NaN,0,0,113798,31.0000,NaN,S,0,1
860,861,3,Mr,male,41.00,2,0,350026,14.1083,NaN,S,2,2
435,436,1,Miss,female,14.00,1,2,113760,120.0000,B96 B98,S,3,3
